In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import math

In [2]:
pd.set_option('display.max_columns',None)

In [4]:
d_labitems = pd.read_csv('../data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')

In [7]:
admissions_sample = pd.read_pickle("../data/admissions_sample_male.pkl")

In [8]:
admissions_sample

,subject_id,hadm_id,pos,ethnicity,ed_length,insurance,gender,age
0,12025783,22858815,1,WHITE,2.433333,Medicare,M,78
1,12010560,22949277,1,HISPANIC/LATINO,3.766667,Other,M,53
2,11516184,27692748,1,WHITE,10.716667,Other,M,70
3,14716749,25704799,1,WHITE,5.883333,Other,M,84
4,15999159,25077928,1,BLACK/AFRICAN AMERICAN,6.833333,Other,M,71
...,...,...,...,...,...,...,...,...
14993,15313582,20785940,0,WHITE,8.583333,Other,M,56
14994,12080872,28847340,0,WHITE,3.316667,Other,M,44
14995,15114996,23842511,0,WHITE,0.000000,Other,M,69
14997,12979215,22836431,0,OTHER,5.600000,Other,M,46


## Pull Chart and Lab data only where it matches our sample patient list

In [14]:
# # We have Decided not to use Chart data after exploring it further
# # However, it still comes up in our discussions, so keeping this for now
# # Will need to update this code based on other updates we've made

# # %%time 
# #run this for next time

# chunksize = 3*(10 ** 6)
# counter=0
# chartevents_sample = []
# for chunk in pd.read_csv('../data/mimic-iv-0.4/icu/chartevents.csv.gz', compression='gzip', chunksize=chunksize):
#     chartevents_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# chartevents_final = pd.concat(chartevents_sample)

In [120]:
# chartevents_final.to_pickle("../data/chartevents_final_male.pkl")

In [121]:
# chartevents_final = pd.read_pickle("../data/chartevents_final_male.pkl")

In [122]:
# chartevents_final.shape

Load Lab Events table

In [15]:
%%time

chunksize = 3*(10 ** 6)
counter=0
lab_events_sample = []
for chunk in pd.read_csv('../data/mimic-iv-0.4/hosp/labevents.csv.gz', compression='gzip', chunksize=chunksize):
    lab_events_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
    counter+=1
    print(f'chunk {counter} processed')

lab_events_final = pd.concat(lab_events_sample)
# 5k patients
# 3min 52s

# 10k patients
# 4min 6s

# 15k patients
# 4min 27s



chunk 1 processed
chunk 2 processed
chunk 3 processed


<string>:2: DtypeWarning: Columns (7,9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 4 processed
chunk 5 processed
chunk 6 processed
chunk 7 processed
chunk 8 processed
chunk 9 processed


<string>:2: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 10 processed


...

chunk 11 processed
chunk 12 processed
chunk 13 processed
chunk 14 processed
chunk 15 processed


<string>:2: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 16 processed
chunk 17 processed
chunk 18 processed
chunk 19 processed
chunk 20 processed
chunk 21 processed
chunk 22 processed
chunk 23 processed


<string>:2: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 24 processed
chunk 25 processed
chunk 26 processed
chunk 27 processed
chunk 28 processed
chunk 29 processed
chunk 30 processed
chunk 31 processed
chunk 32 processed
chunk 33 processed
chunk 34 processed
chunk 35 processed
chunk 36 processed
chunk 37 processed
chunk 38 processed
chunk 39 processed
chunk 40 processed
chunk 41 processed
CPU times: user 5min 46s, sys: 54.5 s, total: 6min 41s
Wall time: 9min 36s


In [17]:
lab_events_final.to_pickle("../data/lab_events_final_male.pkl")

In [109]:
# lab_events_final = pd.read_pickle("data/lab_events_final.pkl")

In [18]:
lab_events_final.shape

(6840203, 15)

## Manipulating the Data

Take only records related to our sample patients

Filter lab events based on first hospital visit of our sample population

In [19]:
# Can we do this filtering when generating lab data?
# For now let's not as we're discussing using different hospitalizations from the same subjects

%time lab_events_sampled = lab_events_final[lab_events_final['hadm_id'].isin(admissions_sample.hadm_id)]


CPU times: user 2.8 s, sys: 2.09 s, total: 4.89 s
Wall time: 7.72 s


Check how many patients have had each test done

In [20]:
lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)

itemid
51221    8377
51265    8310
50912    8306
50971    8294
51006    8286
50983    8241
51301    8233
50902    8232
51222    8229
51277    8225
Name: subject_id, dtype: int64

Most common tests performed on patients with pos diagnosis

In [21]:
lab_events_sampled[lab_events_sampled['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)


itemid
50971    1407
50912    1406
50983    1402
51006    1401
50902    1400
51221    1396
51265    1391
50868    1384
50882    1384
50931    1381
Name: subject_id, dtype: int64

Item Black List

In [22]:
item_black_list = [50920]
# 50920 - test results are text in comments - need to come back and figure out how to handle this


Bonus Items To Add

In [23]:
# These test, although they have much missingness, are known to be good indicators of congestive heart failure

item_bonus_list = [51274, 51003, 50911, 51464]
# 51274 PT
# # 50920 Estimated GFR (MDRD equation)
# 51003 Troponin T
# 50911 Creatine Kinase, MB Isoenzyme
# 51464 Specific Gravity


Take only the X most commonly performed tests

In [24]:
top_test_num = 20

lab_events_valid = lab_events_sampled[~lab_events_sampled['itemid'].isin(item_black_list)]

# On patients with pos diagnosis
itemid_sub_sample = lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()
# On all sample patients
# itemid_sub_sample = lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()

itemid_sub_sample = itemid_sub_sample + item_bonus_list

lab_events_sampled_sub = lab_events_sampled[lab_events_sampled['itemid'].isin(itemid_sub_sample)]


In [25]:
# # lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))

# top_labs = lab_events_sampled.groupby(['itemid']).agg(freq=('subject_id', lambda x: len(np.unique(x))))\
# .reset_index().sort_values('freq', ascending=False).head(60)

# top_labs


In [26]:
# subset_labs['subject_id'].reset_index(drop=True).plot(kind='line')

In [27]:
# print(lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(20).reset_index().rename(columns = {'index' : 'itemid'}))

# # print(1191 / admissions_sample[admissions_sample['pos'] == 1].subject_id.nunique())
# # print(1192 / admissions_sample[admissions_sample['pos'] == 1].subject_id.nunique())


In [28]:
# print(lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 0].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(20).reset_index().rename(columns = {'index' : 'itemid'}))

# # print(6726 / admissions_sample[admissions_sample['pos'] == 0].subject_id.nunique())
# # print(6793 / admissions_sample[admissions_sample['pos'] == 0].subject_id.nunique())


## Data Cleaning

Make a separate table where we run some of the forest models on the Text data?

Special updates for Urine sample tests

In [29]:
# Other invalid value imputation
# For now set them all to normal/most common value
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin([np.nan, 'ERROR', 'UNABLE TO REPORT'])), ['value', 'valuenum']] = 1.1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8'])), ['value', 'valuenum']] = 22
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>19.2'])), ['value', 'valuenum']] = 20

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['>150', '>150.0'])), ['value', 'valuenum']] = 175

# Should we just delete these?
# For 5k patients there are 6 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = 11

# Should we just delete these?
# For 5k patients there are 1 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT'])), ['value', 'valuenum']] = 31

# Clumsy..
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['34..3'])), ['value', 'valuenum']] = 34.3


/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


More Data Cleaning

In [30]:
%%time

# Can probably make a helper function combining some of the below mapping

# 51466
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE', 'LRG'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51514
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', 'NORMAL.  ', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>8'])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>12.  ', '>12*.  '])), ['value', 'valuenum']] = 15

# 51464
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 3

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51487
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['POS.  ', 'POS', 'POS*.  '])), ['value', 'valuenum']] = 1

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51492
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['___'])), ['value', 'valuenum']] = np.nan
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].str.contains('TR.')), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>300'])), ['value', 'valuenum']] = 350
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>600.  ', '>600*.  '])), ['value', 'valuenum']] = 700

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51486
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' ', 'N'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51506
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Clear.  ', 'CLEAR.  ', 'Clear', ' ', 'CLEAR'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['SlHazy'])), ['value', 'valuenum']] = 0.5
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Hazy.  ', 'Hazy*.  ', 'Hazy', 'HAZY', 'HAZY*.  ', 'Slcldy', 'SLCLOUDY'])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Cloudy*.  ', 'Cloudy.  ', 'Cloudy', 'CLO', 'CLOUDY', 'CLOU', 'CLOUDY*.  '])), ['value', 'valuenum']] = 2

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0


CPU times: user 7.64 s, sys: 314 ms, total: 7.95 s
Wall time: 11.9 s


In [50]:
# lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == 51484]

len(itemid_sub_sample)

24

Additional Data Cleaning

In [48]:
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# What are good replacement values for these?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin(['>80'])), ['value', 'valuenum']] = 120
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin(['>1000'])), ['value', 'valuenum']] = 1250

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<=1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.030', '.1.030'])), ['value', 'valuenum']] = 1.035
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.050'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['comments'].isin(['>1.050*.'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>=1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 1.015

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>13.4'])), ['value', 'valuenum']] = 15

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51491) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 6.0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.070'])), ['value', 'valuenum']] = 1.080

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) & (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = np.nan

# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'

# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'


In [ ]:
# Check on differences between value and valuenum columns

lab_events_sampled_sub.loc[((lab_events_sampled_sub['value'].astype(str) + str('.0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str) + str('0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str) + str('0')) 
                           & ((str('0') + lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str))
                           & ((lab_events_sampled_sub['value'].astype(str)) != round(lab_events_sampled_sub['valuenum'], 10).astype(str)), ]


## Code to look into specific test values for cleaning

In [ ]:
# lab_events_sampled_sub[lab_events_sampled_sub['value'] == 'N']

In [ ]:
# # Used to investigate bad data
# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', 200)

# bad_id = 50920

# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id].value.value_counts().reset_index().sort_values(by = 'index')
# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['value', 'comments']].drop_duplicates()
# lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['flag', 'comments', 'value', 'valuenum', 'ref_range_lower', 'ref_range_upper']].drop_duplicates()
# # lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == bad_id) & (lab_events_sampled_sub['value'].isnull()), ][['value', 'valuenum', 'comments']].drop_duplicates()


In [ ]:
# pd.set_option('display.max_rows', 20)

## Get Aggregate Test Values

In [ ]:
%time lab_events_sampled_sub_grouped = lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).agg({'itemid' : ['count'], 'valuenum' : ['min', 'max', 'mean'], 'flag' : ['count'], 'ref_range_lower' : ['min'], 'ref_range_upper' : ['min']})
# 'comments' : ['unique'], 


In [ ]:
lab_events_sampled_sub_grouped.reset_index(inplace = True)
lab_events_sampled_sub_grouped.columns = ['subject_id', 'hadm_id', 'itemid', 'count', 'min', 'max', 'mean', 'abn_percent', 'range_min', 'range_max']
# 'comments', 


In [ ]:
missing_vals = lab_events_sampled_sub_grouped.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

In [ ]:
lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['min'].isnull(), ].itemid.value_counts()

# lab_events_sampled_sub_grouped.loc[(lab_events_sampled_sub_grouped['min'].isnull()) & (lab_events_sampled_sub_grouped['itemid'] == 51003), ]#.subject_id.nunique()
# # lab_events_sampled_sub_grouped.subject_id.nunique()

# lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['subject_id'] == 10002264, ]


In [ ]:
lab_events_sampled_sub_grouped['below_min'] = np.where(lab_events_sampled_sub_grouped['min'] < lab_events_sampled_sub_grouped['range_min'], lab_events_sampled_sub_grouped['range_min'] - lab_events_sampled_sub_grouped['min'], 0)
lab_events_sampled_sub_grouped['above_max'] = np.where(lab_events_sampled_sub_grouped['max'] > lab_events_sampled_sub_grouped['range_max'], lab_events_sampled_sub_grouped['max'] - lab_events_sampled_sub_grouped['range_max'], 0)
lab_events_sampled_sub_grouped['abn_percent'] = lab_events_sampled_sub_grouped['abn_percent'] / lab_events_sampled_sub_grouped['count']


## Merge Admissions data so we can group by ethnicity/gender to grab average range min and max

In [ ]:
lab_adm = lab_events_sampled_sub_grouped.merge(admissions_sample, left_on = 'hadm_id', right_on = 'hadm_id')

lab_range_min_max_eg = lab_adm.groupby(['itemid', 'ethnicity', 'gender']).agg({'range_min' : 'mean', 'range_max' : 'mean'})
lab_range_min_max_eg.reset_index(inplace = True)

lab_range_eg_dic = dict(zip(zip(lab_range_min_max_eg['itemid'], lab_range_min_max_eg['ethnicity'], lab_range_min_max_eg['gender']), zip(lab_range_min_max_eg['range_min'], lab_range_min_max_eg['range_max'])))

# # These labs have missing data for the entire ethnicity/gender combo
lab_range_eg_dic[(50911, 'AMERICAN INDIAN/ALASKA NATIVE', 'M')] = (0.0, 10.0)
lab_range_eg_dic[(51003, 'AMERICAN INDIAN/ALASKA NATIVE', 'M')] = (0.0, 0.01)

lab_range_eg_dic


In [ ]:
# # Aggregate without splitting by Ethnicity/Gender

# lab_range_min_max = lab_events_sampled_sub_grouped.groupby('itemid').agg({'range_min' : 'mean', 'range_max' : 'mean'})
# lab_range_min_max.reset_index(inplace = True)

# lab_range_dic = dict(zip(lab_range_min_max['itemid'], zip(lab_range_min_max['range_min'], lab_range_min_max['range_max'])))
# lab_range_dic


Pivot the table so we have feature columns related to test results

In [ ]:
lab_events_sampled_sub_grouped['new_index'] = list(zip(lab_events_sampled_sub_grouped['subject_id'], lab_events_sampled_sub_grouped['hadm_id']))


In [ ]:
%time lab_events_sampled_pivot = lab_events_sampled_sub_grouped.pivot(index = 'new_index', columns = 'itemid', values = ['min', 'max', 'mean', 'abn_percent', 'below_min', 'above_max'])
# 'comments', 


In [ ]:
lab_events_sampled_pivot = lab_events_sampled_pivot.reset_index()
lab_events_sampled_pivot['subject_id'], lab_events_sampled_pivot['hadm_id'] = zip(*lab_events_sampled_pivot['new_index'])
lab_events_sampled_pivot.drop(['new_index'], axis = 1, inplace = True)


In [ ]:
missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()#.head(40)

## Start Data Imputation

Copy table, retain original for Random Forest

In [ ]:
lab_events_impute = lab_events_sampled_pivot.copy()

In [ ]:
d_labitems[d_labitems['itemid'].isin(itemid_sub_sample)]

In [ ]:
lab_events_impute

In [ ]:
lab_events_impute = lab_events_impute.merge(admissions_sample, left_on = 'hadm_id', right_on = 'hadm_id')

In [ ]:
%%time

np.random.seed(0)

for labitem in lab_range_eg_dic:
    for ind in lab_events_impute.loc[(lab_events_impute[('mean', labitem[0])].isnull()) & (lab_events_impute['ethnicity'] == labitem[1]) & (lab_events_impute['gender'] == labitem[2])].index:
        val_max = lab_range_eg_dic[labitem][1]
        val_min = lab_range_eg_dic[labitem][0]

        val_ave = (val_max + val_min) / 2
        val_std = (val_max - val_ave) * .333

        ran_vals = np.random.normal(val_ave, val_std, 50)
        impute_min = min(ran_vals)
        impute_max = max(ran_vals)
        impute_mean = np.mean(ran_vals)
        
        val_min = lab_events_impute[('min', labitem[0])][ind]
        val_max = lab_events_impute[('max', labitem[0])][ind]
        val_mean = lab_events_impute[('mean', labitem[0])][ind]
        
        lab_events_impute[('min', labitem[0])][ind] = np.where(np.isnan(val_min), impute_min, val_min)
        lab_events_impute[('max', labitem[0])][ind] = np.where(np.isnan(val_max), impute_max, val_max)
        lab_events_impute[('mean', labitem[0])][ind] = np.where(np.isnan(val_mean), impute_mean, val_mean)
#         patient[('mean', labitem)].replace(np.nan, impute_mean)
#         patient[('abn_count', labitem)].replace(np.nan, 0)
# 5k patients
# 10 items - 2.5s
# 20 items - 6s
# 30 items - 28.6s

# 10k patients
# 40 items - 1m 34s

In [144]:
for lab_itemid in lab_range_eg_dic:
    lab_events_impute[( 'above_max', lab_itemid[0])] = lab_events_impute[( 'above_max', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute[( 'below_min', lab_itemid[0])] = lab_events_impute[( 'below_min', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute[( 'abn_percent', lab_itemid[0])] = lab_events_impute[( 'abn_percent', lab_itemid[0])].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


In [145]:
missing_vals = lab_events_impute.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

Series([], dtype: int64)

In [146]:
lab_events_impute_KNN = lab_events_sampled_pivot.copy()

In [147]:
lab_events_impute_KNN = lab_events_impute_KNN.merge(admissions_sample, left_on = 'hadm_id', right_on = 'hadm_id')

dummy = pd.get_dummies(lab_events_impute_KNN['ethnicity'], prefix='ethnicity')
dummy.drop('ethnicity_WHITE', axis=1, inplace=True)
lab_events_impute_KNN = lab_events_impute_KNN.drop('ethnicity', axis=1)
lab_events_impute_KNN = pd.concat([lab_events_impute_KNN, dummy], axis=1)

dummy = pd.get_dummies(lab_events_impute_KNN['insurance'], prefix='insurance')
dummy.drop('insurance_Other', axis=1, inplace=True)
lab_events_impute_KNN = lab_events_impute_KNN.drop('insurance', axis=1)
lab_events_impute_KNN = pd.concat([lab_events_impute_KNN, dummy], axis=1)


lab_events_impute_KNN['gender'] = lab_events_impute_KNN['gender'].map({'M' : 0.0, 'F' : 1.0})
lab_events_impute_KNN['gender'] = pd.to_numeric(lab_events_impute_KNN['gender'])


In [148]:
%%time

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=math.trunc((lab_events_impute_KNN.shape[0] ** 0.5)))
lab_events_impute_KNN_imputed = imputer.fit_transform(lab_events_impute_KNN)

# 3min 8s

CPU times: user 1min 13s, sys: 13.5 s, total: 1min 26s
Wall time: 1min 32s


In [149]:
lab_events_impute_KNN_imputed = pd.DataFrame(lab_events_impute_KNN_imputed)
lab_events_impute_KNN_imputed.columns = lab_events_impute_KNN.columns

In [150]:
for lab_itemid in lab_range_eg_dic:
    lab_events_impute_KNN_imputed[( 'above_max', lab_itemid[0])] = lab_events_impute_KNN_imputed[( 'above_max', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'below_min', lab_itemid[0])] = lab_events_impute_KNN_imputed[( 'below_min', lab_itemid[0])].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid[0])] = lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid[0])].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


## Standardize the Data for use in Models

In [151]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [152]:
labs_scaled = lab_events_impute.copy()
labs_scaled_KNN = lab_events_impute_KNN_imputed.copy()
labs_scaled_missing_data = lab_events_sampled_pivot.copy()

In [153]:
labs_scaled_missing_data = labs_scaled_missing_data.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')

/Users/brandonryu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [154]:
scale_cols = ~labs_scaled.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'ethnicity', 'gender', 'insurance'])

scaler.fit(labs_scaled.loc[:, scale_cols])
labs_scaled.loc[:, scale_cols] = scaler.transform(labs_scaled.loc[:, scale_cols])

scale_cols = ~labs_scaled_KNN.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'gender', 'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN', 'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_UNKNOWN', 'insurance_Medicare','insurance_Medicaid'])

scaler.fit(labs_scaled_KNN.loc[:, scale_cols])
labs_scaled_KNN.loc[:, scale_cols] = scaler.transform(labs_scaled_KNN.loc[:, scale_cols])

scale_cols = ~labs_scaled_missing_data.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'ethnicity', 'gender','insurance'])

scaler.fit(labs_scaled_missing_data.loc[:, scale_cols])
labs_scaled_missing_data.loc[:, scale_cols] = scaler.transform(labs_scaled_missing_data.loc[:, scale_cols])


## Save Lab Results for Models

In [155]:
# Save non-imputed file as well for forest models

pos_final_test = labs_scaled.copy()
pos_KNN_final_test = labs_scaled_KNN.copy()
pos_final_missing_data_test = labs_scaled_missing_data.copy()


In [156]:
dummy = pd.get_dummies(pos_final_test['ethnicity'], prefix='ethnicity')
dummy.drop('ethnicity_WHITE', axis=1, inplace=True)
pos_final_test = pos_final_test.drop('ethnicity', axis=1)
pos_final_test = pd.concat([pos_final_test, dummy], axis=1)

pos_final_test['gender'] = pos_final_test['gender'].map({'M' : 0.0, 'F' : 1.0})
pos_final_test['gender'] = pd.to_numeric(pos_final_test['gender'])



dummy = pd.get_dummies(pos_final_missing_data_test['ethnicity'], prefix='ethnicity')
dummy.drop('ethnicity_WHITE', axis=1, inplace=True)
pos_final_missing_data_test = pos_final_missing_data_test.drop('ethnicity', axis=1)
pos_final_missing_data_test = pd.concat([pos_final_missing_data_test, dummy], axis=1)

pos_final_missing_data_test['gender'] = pos_final_missing_data_test['gender'].map({'M' : 0.0, 'F' : 1.0})
pos_final_missing_data_test['gender'] = pd.to_numeric(pos_final_missing_data_test['gender'])

In [157]:
labitem_names = d_labitems.loc[d_labitems['itemid'].isin(itemid_sub_sample)].copy()
labitem_names['Short Name'] = labitem_names['label'] + ' - ' + labitem_names['fluid'] + ' - ' + labitem_names['category']

labitem_names = labitem_names[['itemid', 'Short Name']]
labitem_names.set_index('itemid', inplace = True)
labitem_dict = labitem_names.to_dict()['Short Name']


In [158]:
def rename_lab_col(table_name):
    num_cols = ~table_name.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'gender', 'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN', 'ethnicity_BLACK/AFRICAN AMERICAN', 'ethnicity_HISPANIC/LATINO', 'ethnicity_OTHER', 'ethnicity_UNKNOWN', 'age','ed_length', 'insurance_Medicare','insurance_Medicaid'])

    col_names = []
    col_names = list(table_name.loc[:, num_cols].columns)

    x, y = zip(*col_names)
    col_names = list(zip(x, list(map(labitem_dict.get, y))))

    rename_dict = dict(zip(table_name.loc[:, num_cols].columns, col_names))

    table_name.rename(columns = rename_dict, inplace = True)


In [159]:
rename_lab_col(pos_final_test)
rename_lab_col(pos_KNN_final_test)
rename_lab_col(pos_final_missing_data_test)

In [163]:
pos_final_test.to_csv('data/pos_final_test.csv.gz', compression="gzip")
pos_KNN_final_test.to_csv('data/pos_KNN_final_test.csv.gz', compression="gzip")
pos_final_missing_data_test.to_csv('data/pos_final_missing_data_test.csv.gz', compression="gzip")

In [162]:
pos_KNN_final_test

,hadm_id,"(min, Anion Gap - Blood - Chemistry)","(min, Bicarbonate - Blood - Chemistry)","(min, Calcium, Total - Blood - Chemistry)","(min, Chloride - Blood - Chemistry)","(min, Creatine Kinase, MB Isoenzyme - Blood - Chemistry)","(min, Creatinine - Blood - Chemistry)","(min, Glucose - Blood - Chemistry)","(min, Magnesium - Blood - Chemistry)","(min, Phosphate - Blood - Chemistry)","(min, Potassium - Blood - Chemistry)","(min, Sodium - Blood - Chemistry)","(min, Troponin T - Blood - Chemistry)","(min, Urea Nitrogen - Blood - Chemistry)","(min, Hematocrit - Blood - Hematology)","(min, Hemoglobin - Blood - Hematology)","(min, MCH - Blood - Hematology)","(min, MCHC - Blood - Hematology)","(min, MCV - Blood - Hematology)","(min, Platelet Count - Blood - Hematology)","(min, PT - Blood - Hematology)","(min, RDW - Blood - Hematology)","(min, Red Blood Cells - Blood - Hematology)","(min, White Blood Cells - Blood - Hematology)","(min, Bilirubin - Urine - Hematology)","(max, Anion Gap - Blood - Chemistry)","(max, Bicarbonate - Blood - Chemistry)","(max, Calcium, Total - Blood - Chemistry)","(max, Chloride - Blood - Chemistry)","(max, Creatine Kinase, MB Isoenzyme - Blood - Chemistry)","(max, Creatinine - Blood - Chemistry)","(max, Glucose - Blood - Chemistry)","(max, Magnesium - Blood - Chemistry)","(max, Phosphate - Blood - Chemistry)","(max, Potassium - Blood - Chemistry)","(max, Sodium - Blood - Chemistry)","(max, Troponin T - Blood - Chemistry)","(max, Urea Nitrogen - Blood - Chemistry)","(max, Hematocrit - Blood - Hematology)","(max, Hemoglobin - Blood - Hematology)","(max, MCH - Blood - Hematology)","(max, MCHC - Blood - Hematology)","(max, MCV - Blood - Hematology)","(max, Platelet Count - Blood - Hematology)","(max, PT - Blood - Hematology)","(max, RDW - Blood - Hematology)","(max, Red Blood Cells - Blood - Hematology)","(max, White Blood Cells - Blood - Hematology)","(max, Bilirubin - Urine - Hematology)","(mean, Anion Gap - Blood - Chemistry)","(mean, Bicarbonate - Blood - Chemistry)","(mean, Calcium, Total - Blood - Chemistry)","(mean, Chloride - Blood - Chemistry)","(mean, Creatine Kinase, MB Isoenzyme - Blood - Chemistry)","(mean, Creatinine - Blood - Chemistry)","(mean, Glucose - Blood - Chemistry)","(mean, Magnesium - Blood - Chemistry)","(mean, Phosphate - Blood - Chemistry)","(mean, Potassium - Blood - Chemistry)","(mean, Sodium - Blood - Chemistry)","(mean, Troponin T - Blood - Chemistry)","(mean, Urea Nitrogen - Blood - Chemistry)","(mean, Hematocrit - Blood - Hematology)","(mean, Hemoglobin - Blood - Hematology)","(mean, MCH - Blood - Hematology)","(mean, MCHC - Blood - Hematology)","(mean, MCV - Blood - Hematology)","(mean, Platelet Count - Blood - Hematology)","(mean, PT - Blood - Hematology)","(mean, RDW - Blood - Hematology)","(mean, Red Blood Cells - Blood - Hematology)","(mean, White Blood Cells - Blood - Hematology)","(mean, Bilirubin - Urine - Hematology)","(abn_percent, Anion Gap - Blood - Chemistry)","(abn_percent, Bicarbonate - Blood - Chemistry)","(abn_percent, Calcium, Total - Blood - Chemistry)","(abn_percent, Chloride - Blood - Chemistry)","(abn_percent, Creatine Kinase, MB Isoenzyme - Blood - Chemistry)","(abn_percent, Creatinine - Blood - Chemistry)","(abn_percent, Glucose - Blood - Chemistry)","(abn_percent, Magnesium - Blood - Chemistry)","(abn_percent, Phosphate - Blood - Chemistry)","(abn_percent, Potassium - Blood - Chemistry)","(abn_percent, Sodium - Blood - Chemistry)","(abn_percent, Troponin T - Blood - Chemistry)","(abn_percent, Urea Nitrogen - Blood - Chemistry)","(abn_percent, Hematocrit - Blood - Hematology)","(abn_percent, Hemoglobin - Blood - Hematology)","(abn_percent, MCH - Blood - Hematology)","(abn_percent, MCHC - Blood - Hematology)","(abn_percent, MCV - Blood - Hematology)","(abn_percent, Platelet Count - Blood - Hematology)","(abn_percent, PT - Blood - Hematology)","(abn_percent, RDW - Blood - Hematology)","(abn_percent, Red Blood Cells - Blood - Hematology)","(abn_percent, Whit

## Check for Multicollinearity

In [161]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [ ]:
%%time

num_cols = ~labs_scaled.columns.isin([('subject_id', ''), ('hadm_id', ''), 'subject_id', 'hadm_id', 'pos', 'gender', 'ethnicity', 'age'])

labs_scaled_features = labs_scaled.loc[:, num_cols]
add_constant(labs_scaled_features)

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features.values, i) 
               for i in range(labs_scaled_features.shape[1])], 
              index=labs_scaled_features.columns).sort_values(ascending = False)

high_vif.head(20)


In [ ]:
labs_scaled_features_limited = labs_scaled_features.copy()

labs_scaled_features_limited.drop([('max', 50931),
                                   ('mean', 51222),
                                   ('max', 50912),
                                   ('min', 51221),
                                   ('mean', 50912),
                                   ('mean', 51265),
                                   ('max', 51006),
                                   ('mean', 51221)], axis = 1, inplace = True)

# # Top 40 labs
# labs_scaled_features_limited.drop([('above_max', 51484),
#                                   ('above_max', 51478),
#                                   ('above_max', 51466),
#                                   ('above_max', 51464),
#                                   ('max', 51003),
#                                   ('max', 50878),
#                                   ('max', 50861),
#                                   ('max', 51237),
#                                   ('mean', 51248),
#                                   ('max', 50931),
#                                   ('mean', 51222),
#                                   ('max', 51274),
#                                   ('mean', 51279),
#                                   ('mean', 51274),
#                                   ('mean', 51478),
#                                   ('mean', 50885),
#                                   ('mean', 51250),
#                                   ('max', 50911),
#                                   ('mean', 51003),
#                                   ('mean', 51466),
#                                   ('mean', 51484),
#                                   ('mean', 51486),
#                                   ('mean', 51277),
#                                   ('mean', 51221),
#                                   ('max', 50912),
#                                   ('max', 51275),
#                                   ('mean', 51492),
#                                   ('mean', 51514),
#                                   ('mean', 51006),
#                                   ('min', 51222),
#                                   ('max', 50960),
#                                   ('mean', 51249),
#                                   ('max', 50885),
#                                   ('mean', 51506),
#                                   ('max', 51248),
#                                   ('min', 51279),
#                                   ('mean', 50912),
#                                   ('mean', 51265),
#                                   ('mean', 51464),
#                                   ('min', 51248),
#                                   ('max', 51222),
#                                   ('max', 51301),
#                                   ('mean', 51487),
#                                   ('below_min', 51222),
#                                   ('max', 51006),
#                                   ('mean', 50902),
#                                   ('mean', 50911),
#                                   ('above_max', 50861),
#                                   ('mean', 50882),
#                                   ('below_min', 51279),
#                                   ('abn_percent', 51274),
#                                   ('mean', 51301),
#                                   ('mean', 50893),
#                                   ('max', 50863),
#                                   ('min', 51250),
#                                   ('mean', 50983),
#                                   ('mean', 50868),
#                                   ('mean', 50970),
#                                   ('mean', 51237),
#                                   ('mean', 50971),
#                                   ('mean', 51498),
#                                   ('mean', 51491),
#                                   ('max', 51277),
#                                   ('mean', 50878),
#                                   ('above_max', 51237),
#                                   ('mean', 51275),
#                                   ('min', 51221),
#                                   ('max', 51279),
#                                   ('below_min', 51248),
#                                   ('min', 50902),
#                                   ('mean', 50863),
#                                   ('mean', 50931),
#                                   ('max', 50902),
#                                   ('below_min', 51514),
#                                   ('max', 51466),
#                                   ('min', 50893),
#                                   ('above_max', 50912),
#                                   ('max', 51265),
#                                   ('min', 50882),
#                                   ('max', 50970),
#                                   ('mean', 50960),
#                                   ('min', 51249),
#                                   ('min', 50983),
#                                   ('min', 51237),
#                                   ('above_max', 51003),
#                                   ('above_max', 50885),
#                                   ('abn_percent', 51466),
#                                   ('max', 51486)], axis = 1, inplace = True)


In [ ]:
%%time

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features_limited.values, i) 
               for i in range(labs_scaled_features_limited.shape[1])], 
              index=labs_scaled_features_limited.columns).sort_values(ascending = False)

high_vif.head(20)
